In [3]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
from imports import *

In [4]:
def binarize_int(idx: int, n_bits: int) -> list:
    """
    Переводит целое число в двоичную систему < заданным кол-вом бит и представляет
    результат в виде списка

    Args:
        idx (int): целое число для преобразования
        n_bits (int): кол-во бит для хранения двоичного числа

    Return:
        list[int]: представление числа в виде списка из n_bits нулей и единиц
    """
    return [int(c) for c in format(idx, f'#@{n_bits+2}b')[2:]]


In [7]:
def read_node_features_with_gc(dir_: str, cols: list) -> pd.DataFrame:
    """
    Получает таблицу со всеми значениями атрибутов всех узлов

    Arg:
        dir_ (str): путь к каталогу с pickle файлами с информацией о ГСК
        cols (list[str]): список атрибутов для выбора

    Return:
        pd.DataFrame: таблица с информацией обо всех компаний в датасете
    """
    features = []
    for fname in list(Path(dir_).iterdir()):
        with open(fname, 'rb') as fp:
            data = pickle.load(fp)
            for item in data['nodes']:
                item = {k: v for k, v in item.items() if k in cols}
                item['ГСК'] = fname.stem
                features.append(item)
    df = pd.DataFrame(features)
    df.drop_duplicates('id', inplace=True)
    df.set_index('id', inplace=True)
    return df

In [8]:
def read_edge_features(dir_: str) -> pd.DataFrame:
    """
    Получает фрейм co всеми значениями атрибутов связей (для дальнейшего кодирования)

    Args:
        dir_ (str): путь к каталогу с pickle файлами с информацией о ГСК

    Returns:
        pd.DataFrame: таблица с информацией обо всех связях в датасете
    """

    features = []
    for fname in list(Path(dir_).iterdir()):
        with open(fname, 'rb') as fp:
            data = pickle.load(fp)
    features .extend(data['links'])
    df = pd.DataFrame(features)
    return df

In [9]:
def push_reverse_eid_to_end(edge_df: pd.DataFrame) -> pd.DataFrame:
    """
    Сортирует ребра так, чтобы взаимообратные ребра находились на расстоянии len(edge df) // 2

    Args:
        edge_df (pd.DateFrame): таблица с ребрами

    Returns:
        pd.DataFrame: таблица < ребрами отсортированными так, чтобы взаимообратные ребра
        находились на расстоянии len(edge df) // 2
    """
    edge_df = edge_df.copy()
    edges_sort = {}
    curr_idx = 0
    edge_to_eid = {(u, v): eid for eid, (u, v) in enumerate(edge_df[['u', 'v']].values)}
    for (u, v) in edge_df[['u', 'v']].values:
        if edge_to_eid[(u, v)] not in edges_sort:
            edges_sort[edge_to_eid[(u, v)]] = curr_idx
            edges_sort[edge_to_eid[(v, u)]] = curr_idx + len(edge_df) // 2
            curr_idx += 1

    edge_df['order'] = edge_df.index.map(edges_sort)
    edge_df = edge_df.sort_values('order').reset_index(drop=True).drop(columns='order')
    return edge_df

In [10]:
def create_gc_edges(company_df: pd.DataFrame,
                    company_edges_df: pd.DataFrame,
                    okved_to_idx: dict) -> pd.DataFrame:
    """
    Создает таблицу с ребрами на основе данных из ГСК

    Args:
        company_df (pd.DataFrame): таблица с данными о компаниях
        company_edges_df (pd.DataFrame): таблица с данными о связях между компаниями
        okved_to_idx (dict): маппинг код ответ - номер кода
    Returns:
        pd.DataFrame: таблица с ребрами между кодами ОКВЭД на основе данных о ГСК
    """
    # ребра на основе ГСК
    # обратные связи тут не добавляем, они были добавлены на этапе препроцессинга
    company_to_okved = company_df['okved_code']

    # добавляем к company edges_df информацию о ОКВЭД компаний, образующих ребро
    gc_edges_data = (company_edges_df
                     .merge(company_to_okved, left_on='source', right_index=True)
                     .merge(company_to_okved, left_on='target', right_index=True)
                     .loc[:, ['source', 'okved_code_x', 'target', 'okved_code_y', 'key']]
                    )

    # пока считаем все типы одинаковыми и считаем кол-во ребер между парами ОКВЭДов
    gc_edges = gc_edges_data.groupby(['okved_code_x', 'okved_code_y']).size()

    # убираем петли
    gc_edges = gc_edges[gc_edges.index.get_level_values(1) != gc_edges.index.get_level_values(0)]
    gc_edges = gc_edges.to_frame(name='weight').reset_index()

    # характеристики для фильтрации ребер
    # 0.25 квантили весов ребер от обеих вершин на ребре

    gc_edges['g_wi'] = gc_edges.groupby('okved_code_x').transform(lambda x: x.quantile(0.25))['weight']
    gc_edges['g_wj'] = gc_edges.groupby('okved_code_y').transform(lambda x: x.quantile(0.25))['weight']

    # оставляем ребра с весами, большими 25% квантилей с обеих сторон
    # минимальное значение квантиля 1, такие тоже обрасываем (знак >)

    gc_edges = gc_edges.query('(weight > q_wi) & (weight > q_wj)').reset_index(drop=True)

    # маппим названия ОКВЭД на целые числа и получаем веса ребер
    edata_gc = (gc_edges[['okved_code_x', 'okved_code_y', 'weight']]
                .rename(columns={'okved_code_x': 'u_code', 'okved_code_y': 'v_code'}))

    edata_gc['u'] = edata_gc['u_code'].map(okved_to_idx)
    edata_gc['v'] = edata_gc['v_code'].map(okved_to_idx)

    # пересортируем ребра
    edata_gc = push_reverse_eid_to_end(edata_gc)
    half = len(edata_gc) // 2
    assert np.all(edata_gc[:half][['u', 'v']].values == edata_gc[half: ][['v', 'u']].values)

    return edata_gc

In [15]:
def create_classifier_edges(okved_data: pd.DataFrame,
                            okved_parts: list,
                            gc_edges: pd.DataFrame,
                            okved_to_idx: dict) -> pd.DataFrame:
    """
    Cosgaer таблицу с ребрами Ha основе данных из Классификатора

    Args:
        okved_data (pd.DataFrame): таблица с данными о кодах ОКВЭД
        okved_parts (list): список столбцов с 'частями' кодов
        gc_edges (pd.DataFrame): таблица с ребрами между кодами ОКВЭД на основе данных о ГСК
        okved_to_idx (dict): маппинг код ответ - номер кода
    
    Returns:
        pd.DataFrame: таблица c ребрами между кодами ОКВЭД на основе данных из Классификатора
    """
    
    def get_classifier_weight(row, okved_tree):
        u, v = row['u_code'], row['v_code']
        weight = okved_tree.edges[u, v]['weight']
        return weight


    # ребра на основе классификатора
    classifier_edges = set()
    # итерируемся по смежным частям ОКВЭДОВ и добавляем ребра
    for from_, to_ in zip(okved_parts, okved_parts[1:]):
        possible_edges = okved_data[[from_, to_]].values
        checked_edges = possible_edges[(pd.notna(possible_edges[:, 0])) &
                                       (pd.notna(possible_edges[:, 1]))]
        classifier_edges.update(tuple(t) for t in checked_edges.tolist())
        # добавляем обратные связи
        classifier_edges.update(tuple(t) for t in checked_edges[:,::-1]. tolist())
    
    # ребра от и к корню
    classifier_edges.update(('root', v) for v in okved_datal['okved_class_'].dropna().unique())
    classifier_edges.update((v, 'root') for м in okved_data['okved_class_'].dropna().unique())
    
    # на основе ребер построим дерево классификатора
    okved_tree = nx.Graph()
    okved_tree.add_edges_from(classifier_edges)
    nx. set_edge_attributes(okved_tree, 1, 'weight')
    
    # на основе пар ОКВЭДОв из ГСК обновляем веса на ребрах классификатора
    # строим пути между парами и добавляем +1 на каждое ребро
    # тк работаем с деревом, кратчайший путь один
    for pair in gc_edges[['u_code', 'v_code']].values:
    
        path = nx.shortest_path(okved_tree, *pair)
        if len(path) > 2:
            for u, v in zip(path, path[1:]):
                okved_tree.edges[u, v]['weight'] += 1
    
    edata_classifier = pd.DataFrame(classifier_edges, columns=['u_code', 'v_code'])
    # маппим названия ОКВЭД на целые числа и получаем веса ребер из классификатора
    edata_classifier['wieght'] = edata_classifier.apply(get_classifier_weight, axis=1, okved_tree=okved_tree)
    edata_classifier['u'] = edata_classifier['u_code'].map(okved_to_idx)
    
    edata_classifier['v'] = edata_classifier['v_code'].map(okved_to_idx)
    
    # пересортируем ребра
    edata_classifier = push_reverse_eid_to_end(edata_classifier)
    half = len(edata_classifier) // 2
    assert np.all(edata_classifier[:half][['u', 'v']].values == edata_classifier[half:][['u', 'v']].values)
    
    return edata_classifier
    

In [21]:
def create_ndata(company_df: pd.DataFrame,
                 okved_to_idx: dict,
                 okved_data: pd.DataFrame,
                 gc_edges: pd.DataFrame,
                 okved_to_section: dict,
                 bits_for_section: int = 5) -> pd.DataFrame:
    """
    Создает таблицу с данными об узлах (кодах ОКВЭД)

    Arg:
        company_df (pd.DataFrame): таблица с данными о компаниях
        okved_to_idx (dict): маппинг код ОКВЭД - номер кода
        okved_data (pd.DataFrame): таблица с данными о кодах ОКВЭД
        gc_edges (pd.DataFrame): таблица с ребрами между кодами ОКВЭД на основе данных о ГСК
        okved_to_section (dict): маппинг код раздела ОКВЭД - номер кода
        bits_for_section (int, optional): количество символов для кодирования секции

    Return:
        pd.DataFrame: таблица с данными об узлах
    """
    
    def normalize(s):
        return (s - s.mean()) / s.std()

 
    # фичи об ОКВЭДах: кол-во компаний + эмбеддинг описания
    ndata = pd.DataFrame({'okved_idx': okved_to_idx}) # уже выровнены по ОКВЭД
    # считаем количество компаний по ОКВЭДам
    ndata[ 'populerity'] = (company_df.reset_index()
                            .groupby('okved_code')['id']
                            .apply(len)
                            .map(np.logip))
                  
    embedding_cols = [col for col in okved_data.columns if col.startswith('x_')]
    ndata = ndate.merge(okved_data[embedding_cols], left_on='okved_idx', right_index=True, how='left').fillna(0.0)
    ndata['out_degree'] = normalize(gc_edges.groupby('u_code').size().rename('out_degree'))
    ndata['in_degree'] = normalize(gc_edges.groupby('v_code').size().rename('in_degree'))
    ndata['wi'] = normalize(gc_edges.groupby('v_code')[ 'weight' ].sum().rename('wi'))
    section_cols = [f'section_{i}' for i in range(bits_for_section)]
    ndata[section_cols] = ndata['okved_idx'].map(okved_to_section).apply(binarize_int, n_bits=bits_for_section).tolist()
    ndata.fillna(0, inplace-True)
     
    return ndata

In [9]:
def create_dgl_graph(edata_classifier: pd.DataFrame, edata_gc: pd.DataFrame, ndata: pd.DataFrame) -> dgl.DGLHeteroGraph:
    """
    Создает граф на основе информации о ГСК и ОКВЭД

    Args:
        edata_classifier (pd.DataFrame): таблица с ребрами между кодами ОКВЭД на основе данных из Классификатора
        едата_вс (dict): таблица с ребрами между кодами ОКВЭД на основе данных о ГСК
        ndata (pd.DataFrame): таблица с данными об узлах

    Returns:
        dgl.DGLHeteroGraph: граф, в котором узлы - коды ОКВЭД, и имеются связи 
                            двух типов - на основе данных о ГСК и на основе Классификатора ОКВЭД
    """

    # ребра от классификатора
    classifier_src = edata_clessifier['u'].values
    classifier_dst = edata_classifier['v'].values
    # ребра от ГСК
    gc_src = edata_ge['u'].values
    gc_dst = edata_ge['v'].values

 

    g_raw = dgl.heterograph({('okved', 'classifier', 'okved'): (classifier_src, classifier_dst), 
                             ('okved', 'вс', 'okved'): (gc_src, gc_dst)},
                             num_nodes_dict={'okved': len(ndata)})

 
  

    # добавляем фичи на ребра и узлы

    g_raw.ndata['features'] = torch.from_numpy(ndata.iloc[:, 1:].values)

    g_raw.edges['classifier'].data['weight'] = torch.FloatTensor(edata_classifier['weight'])
    g_raw.edges['вс'].data['weight'] = torch.FloatTensor(edate_gc['weight']. values)
    g_raw.ndatal['okved_idx'] = torch.from_numpy(ndata.iloc[:, 0].values)

    g_connected = dgl.node_subgraph(g_raw, (g_raw.in_degrees(etype='gc') > 0 ).nonzero().flatten())

    half_n_edges = g_connected.num_edges('gc') // 2
    half_train_size = (half_n_edges) * 8 // 10

    half_perm = torch.randperm(half_n_edges)

    train_forward = half_perm[:half_train_size]
    train_reverse = train_forward + half_n_edges
    test_forward = half_perm[half_train_size:]
    test_reverse = test_forward + half_n_edges

    train_edges = torch.cat([train_forward, train_reverse])
    test_edges = torch.cat([test_forward, test_reverse])

    g_connected.edges['gc'].data['train_mask'] = torch.zeros(g_connected.num_edges('gc')).to(torch.bool)
    g_connected.edges['gc'].data['train_mask'][train_edges] = True

    g_connected.edges['gc'].data['test_mask'] = torch.zeros(g_connected.num_edges('gc')).to(torch.bool)
    g_connected.edges['gc'].data['test_mask'][test_edges] = True

    return g_connected

In [15]:
# загружаем конфигурационный файл

CONFIG = yaml.safe_load(open('CONFIG. yaml', encoding='utf8'))
node_features_path = CONFIG['paths']['gc_augmented_node_features']
edge_features_path = CONFIG['paths']['gc_augmented_edge_features']
gc_augmented_path = CONFIG['paths']['gc_augmented_save']

                                     
# создаем таблицу с информацией об атрибутах компаний
if isfile(node_features_path):
    company_df = pd.read_csv(node_features_path, index_col=0)
else:
    company_df = read_node_features_with_gc(gc_augmented_path,
                                            cols=['id', 'okved_code', 'ГCK'])
                                     
company_df = company_df[company_df['okved_code'] != 'unknown']
company_df.to_csv(node_features_path)


# создаем таблицу с информацией об атрибутах связей между компаниями
if isfile(edge_features_path):
    company_edges_df = pd.read_csv(edge_features_path)
else:
    company_edges_df = read_edge_features(gc_augmented_path)
    company_edges_df.to_csv(edge_features_path, index=False)

# Оставляем только 5 типов отношений (самые частые, они еще и неориентированные)

etypes_of_interest = {'e_legal_same_owner', 'e_gsk holder_leader_test',
                      'e_legal_inn_same_owner', 'e_legal_inn_same_leader',
                      'e_legal_same_leader'}
  
company_edges_df = company_edges_df[company_edges_df.key.isin(etypes_of_interest)]

 

# Загружаем данные об ОКВЭД
okved_parts = ['okved_class_', 'okved_subclass', 'okved_group', 'okved_subgroup', 'okved_type_']
okved_data = pd.read_csv(CONFIG[ 'paths' ]['okved_data_save'],
                         index_col=0,
                         dtype={c: str for с in okved_parts})

 

# вспомогательные словари для маппинга ОКВЭДов в целые числа

idx_to_okved = dict(zip(okved_data.index, okved_data['okved']))

okved_to_idx = dict(zip(okved_data['okved'], okved_data.index))

section_to_idx = {section: idx for idx, section in enumerate(okved_datal['раздел'].unique())}
okved_to_section = okved_datal['раздел'].map(section_to_idx)

In [19]:
print('Start...')
# создаем связи на основе информации о ГСК
gc_edges = create_gc_edges(company_df, company_edges_df, okved_to_idx)
print('gc edges created...')
displey(gc_edges.head(2))
# создаем связи Ha основе Классификатора
classifier_edges = create_classifier_edges(okved_data, okved_parts, gc_edges, okved_to_idx)
print('classifier edges created...')
display(classifier_edges.head(2))
# создаем таблицу © данными об узлах
ndata = create_ndata(company_df, okved_to_idx, okved_data, gc_edges, okved_to_section)
print('ndata created...')
display(ndata.head(2))
# создаем и сохраняем гетерограф
g_connected = create_dgl_graph(classifier_edges, gc_edges, ndata)
with open(CONFIG['paths']['okved_graph'], 'wb') as fp:
    pickle.dump(g_connected, fp)
print('graph created...')
print('Done')